## Load Data

In [46]:
import numpy as np

data = np.load("data/metrla.npy")

In [47]:
data.shape

(34272, 207, 3)

In [48]:
num_steps=12

## Split Data

In [49]:
T, N, _ = data.shape

# Calculate the split points eg. 7:1:2
train_end = int(T * 0.7)
val_end = train_end + int(T * 0.1)

# Split the dataset
train = data[:train_end]
# train = data
test = data[val_end:]

## Get Historal Average

In [50]:
# Create a dictionary to store lists of observations for each (tod, dow) pair
history = {}
# Iterate through the historical data to populate the history dictionary
for i in range(train.shape[0]-num_steps-1):
    
    key = (train[i, 0, 1], train[i, 0, 2])  # (tod, dow) tuple
    if key not in history:
        history[key] = []
        
    # Collect observations for the next num_steps steps
    future_values = train[i+1:i+num_steps+1,:,0]
        
    history[key].append(future_values)  # Collect and store future values


In [51]:
history_averages = {}

# Calculate the average values for each (tod, dow) pair
for k, v in history.items():
    stacked_array = np.stack(v, axis=-1)
    averages = np.mean(stacked_array, axis=-1)
    history_averages[k] = averages

In [52]:
len(history_averages)

2016

## Test Data

In [53]:
from metrics import compute_all_metrics

import torch

preds = []
labels = []

# Retrieve labels and predictions
for i in range(test.shape[0]-num_steps-1):

    key = (test[i, 0, 1], test[i, 0, 2])  # (tod, dow) tuple
    
    labels.append(test[i+1:i+num_steps+1,:,0])
    preds.append(history_averages[key])
    
labels = np.stack(labels, axis=0)
preds = np.stack(preds, axis=0)

In [54]:
preds.shape

(6842, 12, 207)

In [55]:
labels = torch.Tensor(labels)
preds = torch.Tensor(preds)

# handle the precision issue when performing inverse transform to label
mask_value = torch.tensor(0)
if labels.min() < 1:
    mask_value = labels.min()

test_mae = []
test_mape = []
test_rmse = []
print('check mask value', mask_value)

# Calculate metrics
for i in range(12):
    res = compute_all_metrics(preds[:,i,:], labels[:,i,:], mask_value)
    log = 'Horizon {:d}, Test MAE: {:.4f}, Test RMSE: {:.4f}, Test MAPE: {:.4f}'
    print(log.format(i + 1, res[0], res[2], res[1]))
    test_mae.append(res[0])
    test_mape.append(res[1])
    test_rmse.append(res[2])

log = 'Average Test MAE: {:.4f}, Test RMSE: {:.4f}, Test MAPE: {:.4f}'
print(log.format(np.mean(test_mae), np.mean(test_rmse), np.mean(test_mape)))

check mask value tensor(0.)
Horizon 1, Test MAE: 7.1165, Test RMSE: 9.8372, Test MAPE: 0.1689
Horizon 2, Test MAE: 7.1162, Test RMSE: 9.8369, Test MAPE: 0.1689
Horizon 3, Test MAE: 7.1155, Test RMSE: 9.8361, Test MAPE: 0.1689
Horizon 4, Test MAE: 7.1138, Test RMSE: 9.8347, Test MAPE: 0.1689
Horizon 5, Test MAE: 7.1117, Test RMSE: 9.8335, Test MAPE: 0.1688
Horizon 6, Test MAE: 7.1114, Test RMSE: 9.8332, Test MAPE: 0.1688
Horizon 7, Test MAE: 7.1112, Test RMSE: 9.8330, Test MAPE: 0.1688
Horizon 8, Test MAE: 7.1108, Test RMSE: 9.8326, Test MAPE: 0.1688
Horizon 9, Test MAE: 7.1106, Test RMSE: 9.8324, Test MAPE: 0.1688
Horizon 10, Test MAE: 7.1103, Test RMSE: 9.8320, Test MAPE: 0.1688
Horizon 11, Test MAE: 7.1101, Test RMSE: 9.8318, Test MAPE: 0.1688
Horizon 12, Test MAE: 7.1099, Test RMSE: 9.8315, Test MAPE: 0.1688
Average Test MAE: 7.1123, Test RMSE: 9.8338, Test MAPE: 0.1689
